In [1]:
import numpy as np
import pandas as pd

In [2]:
!pip install scikit-network==0.31.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 47.1 MB/s eta 0:00:00


In [2]:
import string

In [3]:
import time

In [1]:
import re

In [3]:
!rm *.tsv.gz
!wget -q https://datasets.imdbws.com/name.basics.tsv.gz
!wget -q https://datasets.imdbws.com/title.principals.tsv.gz
!wget -q https://datasets.imdbws.com/title.basics.tsv.gz
!wget -q https://datasets.imdbws.com/title.akas.tsv.gz
!ls -la

rm: cannot remove '*.tsv.gz': No such file or directory
total 1187320
drwxr-xr-x 1 root root      4096 Nov 20 05:03 .
drwxr-xr-x 1 root root      4096 Nov 20 04:58 ..
drwxr-xr-x 4 root root      4096 Nov 16 14:23 .config
-rw-r--r-- 1 root root 257060653 Nov 18 13:16 name.basics.tsv.gz
drwxr-xr-x 1 root root      4096 Nov 16 14:27 sample_data
-rw-r--r-- 1 root root 320075860 Nov 18 13:16 title.akas.tsv.gz
-rw-r--r-- 1 root root 180588620 Nov 19 13:16 title.basics.tsv.gz
-rw-r--r-- 1 root root 458055960 Nov 19 13:16 title.principals.tsv.gz


In [4]:
# load the titles
title = pd.read_csv('title.basics.tsv.gz', sep='\t', low_memory=True).set_index('tconst')[['titleType','primaryTitle','startYear']]

<ipython-input-4-5fcbdf97272b>:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  title = pd.read_csv('title.basics.tsv.gz', sep='\t', low_memory=True).set_index('tconst')[['titleType','primaryTitle','startYear']]


In [5]:
title.head()

,titleType,primaryTitle,startYear
tconst,,,
tt0000001,short,Carmencita,1894
tt0000002,short,Le clown et ses chiens,1892
tt0000003,short,Pauvre Pierrot,1892
tt0000004,short,Un bon bock,1892
tt0000005,short,Blacksmith Scene,1893


In [6]:
title.dtypes

titleType       object
primaryTitle    object
startYear       object
dtype: object

In [7]:
title.startYear.value_counts()

\N      1384750
2021     471981
2018     439879
2022     439323
2017     435428
         ...   
1889          2
1874          1
1883          1
2031          1
1885          1
Name: startYear, Length: 153, dtype: int64

In [8]:
title_new = title[title['startYear'].str.find("\\N") == -1]

In [9]:
title_new.shape

(8945569, 3)

In [10]:
title_new['startYear'] = title_new['startYear'].values.astype('int')

<ipython-input-10-aa36d257ccea>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  title_new['startYear'] = title_new['startYear'].values.astype('int')


In [11]:
title_new.dtypes

titleType       object
primaryTitle    object
startYear        int64
dtype: object

In [12]:
title_fil = title_new[title_new['startYear'] == 2005] # filter by year

In [13]:
title_fil.head()

,titleType,primaryTitle,startYear
tconst,,,
tt0088751,movie,The Naked Monster,2005
tt0114532,tvEpisode,Erbarmungslos,2005
tt0118141,movie,What Is It?,2005
tt0120667,movie,Fantastic Four,2005
tt0120886,tvMovie,A Tryst with Destiny,2005


In [14]:
title_fil['startYear'].value_counts()

2005    158556
Name: startYear, dtype: int64

In [ ]:
# load the cast of each film
cast  = pd.read_csv('title.principals.tsv.gz', sep = '\t')[['tconst', 'nconst', 'category']]
# only consider actors, not directors, composors etc.
cast = cast[cast.category.isin({'actor','actress'})]
cast.head()

In [ ]:
# only consider movies, not series, etc
movies = title[title['titleType'] == 'movie']
cast = cast[cast['tconst'].isin(movies.index)]
cast.head()

In [ ]:
region = pd.read_csv('title.akas.tsv.gz',sep='\t',usecols=['titleId','region'],low_memory = True,dtype={'region':'str','titleId':'str'}).set_index('titleId')['region']
region.value_counts().head(10)

In [ ]:
region.head()

In [ ]:
name = pd.read_csv('name.basics.tsv.gz', sep = '\t', low_memory=True, na_values = '\\N', dtype={'birthYear': float}).set_index('nconst')[['primaryName', 'birthYear']]
name.head()

In [ ]:
from scipy.sparse import csr_matrix

In [ ]:
def get_pairs(lang=None, min_acted=25, min_pairings=1):
    '''
    Returns an adjacency matrix and actor mapping of actor pairs where:
    - Each actor has acted in at least min_acted films
    - The two actors have acted together in at least min_pairings films
    - And (optionally), belong to a region `lang` (IN, UN, etc)
    '''
    graph = cast
    if lang is not None:
        graph = graph[graph['tconst'].isin(region[region == lang].index)]
        # graph = graph[graph['tconst'].isin(region[region == lang].index)]
    name_freq = graph['nconst'].value_counts()
    top_names = name_freq[name_freq >= min_acted]
    top_actors = graph[graph['nconst'].isin(top_names.index)]

    p = top_actors.copy()
    p['title'] = p['tconst'].astype('category')
    p['name'] = p['nconst'].astype('category')

    row = p['title'].cat.codes.values
    col = p['name'].cat.codes.values
    data = np.ones(len(p), dtype='int')

    matrix = csr_matrix((data, (row, col)))
    square = matrix.T * matrix
    square.setdiag(0)
    square = square.tocoo()

    pairs = pd.DataFrame({
        'row': square.row,
        'col': square.col,
        'n': square.data
    })
    pairs = pairs[pairs.n >= min_pairings].reset_index(drop=True)
    return pairs, name.reindex(p['name'].cat.categories)

def lookup(pairs, cat):
    pairs = pd.concat([
        pairs,
        cat.iloc[pairs.row].reset_index(drop=True),
        cat.iloc[pairs.col].reset_index(drop=True),
    ], axis=1)
    pairs = pairs.drop(columns=['row', 'col'])
    pairs.columns = ['count', 'name1', 'year1', 'name2', 'year2']
    return pairs.sort_values('count', ascending=False)

In [ ]:
pairs, cat = get_pairs(lang='IN', min_acted=1, min_pairings=1) # setting lang = 'IN' will apply a filter allowing only INdian movies

In [ ]:
ForKumu = lookup(pairs, cat)

In [ ]:
ForKumu = ForKumu[['name1', 'name2', 'count']]
ForKumu = ForKumu.rename(columns={'name1':'From',
                                  'name2':'To',
                                  'count':'Strength'})


In [ ]:
ForKumu.head()

,From,To,Strength
49374,Bahadur,Adoor Bhasi,172
71837,Adoor Bhasi,Bahadur,172
161691,Adoor Bhasi,Jayabharati,125
49363,Jayabharati,Adoor Bhasi,125
49354,Prem Nazir,Adoor Bhasi,124


In [ ]:
ForKumu.shape # if the kumu website crashes when you try to upload this massive dataset, try uploading a sample of this, like the first 10,000 rows.

(862080, 3)